In [14]:
from Workbench.transport.QuestClient import QuestDBClient

client = QuestDBClient(host="34.97.186.69",
                       port=8812,
                       read_only=True)

query ="""select * from top_of_book
where exchange = '{}' and symbol = '{}';
"""

df_htx = client.execute_query(query.format("HTX","SOL-USDT"))
df_binance = client.execute_query(query.format("Binance","SOLUSDT"))
df_bb = client.execute_query(query.format("Bybit","SOLUSDT"))

In [15]:
df_htx['mid_price'] = (df_htx['bid_price'] + df_htx['ask_price']) / 2
df_binance['mid_price'] = (df_binance['bid_price'] + df_binance['ask_price']) / 2
df_bb['mid_price'] = (df_bb['bid_price'] + df_bb['ask_price']) / 2
df_htx['spread_bp'] = (df_htx['ask_price'] - df_htx['bid_price']) / df_htx['mid_price'] * 10000
df_binance['spread_bp'] = (df_binance['ask_price'] - df_binance['bid_price']) / df_binance['mid_price'] * 10000
df_bb['spread_bp'] = (df_bb['ask_price'] - df_bb['bid_price']) / df_bb['mid_price'] * 10000

In [16]:
df_binance

,symbol,exchange,bid_price,bid_qty,ask_price,ask_qty,mid_price,spread_bp
timestamp,,,,,,,,
2025-05-25 13:42:53.997,SOLUSDT,Binance,172.21,211.17,172.22,1043.00,172.215,0.580670
2025-05-25 13:42:54.003,SOLUSDT,Binance,172.21,314.90,172.22,1031.18,172.215,0.580670
2025-05-25 13:42:54.023,SOLUSDT,Binance,172.21,281.22,172.22,818.36,172.215,0.580670
2025-05-25 13:42:54.027,SOLUSDT,Binance,172.21,201.25,172.22,818.36,172.215,0.580670
2025-05-25 13:42:54.087,SOLUSDT,Binance,172.21,234.27,172.22,1031.17,172.215,0.580670
...,...,...,...,...,...,...,...,...
2025-05-25 14:11:10.188,SOLUSDT,Binance,172.13,272.90,172.14,298.56,172.135,0.580939
2025-05-25 14:11:10.231,SOLUSDT,Binance,172.13,272.93,172.14,298.56,172.135,0.580939
2025-05-25 14:11:10.260,SOLUSDT,Binance,172.13,272.90,172.14,298.62,172.135,0.580939


In [17]:
df_merged = (
    df_htx[["mid_price", "spread_bp"]].rename(columns=lambda x: f"{x}_htx")
    .join(df_binance[["mid_price", "spread_bp"]].rename(columns=lambda x: f"{x}_binance"), how="outer")
    .join(df_bb[["mid_price", "spread_bp"]].rename(columns=lambda x: f"{x}_bb"), how="outer")
    .sort_index()
)

In [19]:

df_merged.fillna(method="ffill", inplace=True)

/var/folders/y8/5zd6qyks6gqgfm_tfrq0nb6w0000gn/T/ipykernel_90304/3103239665.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged.fillna(method="ffill", inplace=True)


In [20]:
df_merged

,mid_price_htx,spread_bp_htx,mid_price_binance,spread_bp_binance,mid_price_bb,spread_bp_bb
timestamp,,,,,,
2025-05-25 13:42:52.290,NaN,NaN,NaN,NaN,172.255,0.580535
2025-05-25 13:42:52.309,NaN,NaN,NaN,NaN,172.255,0.580535
2025-05-25 13:42:52.330,NaN,NaN,NaN,NaN,172.255,0.580535
2025-05-25 13:42:52.352,NaN,NaN,NaN,NaN,172.255,0.580535
2025-05-25 13:42:52.368,NaN,NaN,NaN,NaN,172.255,0.580535
...,...,...,...,...,...,...
2025-05-25 14:11:12.992,172.2359,-8.256118,172.135,0.580939,172.135,0.580939
2025-05-25 14:11:13.010,172.2359,-8.256118,172.135,0.580939,172.135,0.580939
2025-05-25 14:11:13.032,172.2359,-8.256118,172.135,0.580939,172.135,0.580939
